In [1]:
import os
from dotenv import load_dotenv
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [2]:
prompt = ChatPromptTemplate.from_messages(
    [ ("system", "당신은 개발자입니다.") , 
     ("user", "{input}")]
)
print(prompt)

prompt_text = prompt.format(input="JavaScript는 무엇인가요? 자세하게 설명해주세요.")

input_variables=['input'] input_types={} partial_variables={} messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='당신은 개발자입니다.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={})]


In [3]:
# Groq API를 사용하는 ChatOpenAI 인스턴스 생성
llm = ChatOpenAI(
    api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0.7
)
print(llm)

client=<openai.resources.chat.completions.completions.Completions object at 0x000001F314225E50> async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x000001F314296F60> root_client=<openai.OpenAI object at 0x000001F3114D4530> root_async_client=<openai.AsyncOpenAI object at 0x000001F313D9BC20> model_name='meta-llama/llama-4-scout-17b-16e-instruct' temperature=0.7 model_kwargs={} openai_api_key=SecretStr('**********') openai_api_base='https://api.groq.com/openai/v1'


In [4]:
try:
    response = llm.invoke(prompt_text)
    print(type(response))
    print(response)
    print("응답:", response.content)
except Exception as e:
    print(f"오류 발생: {e}")

오류 발생: Error code: 401 - {'error': {'message': 'Invalid API Key', 'type': 'invalid_request_error', 'code': 'invalid_api_key'}}


### LCEL (Prompt + LLM모델)

In [1]:
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template("You are an expert in AI Expert. Answer the question. <Question>: {input}에 대해 쉽게 설명해주세요.")
prompt

PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='You are an expert in AI Expert. Answer the question. <Question>: {input}에 대해 쉽게 설명해주세요.')

In [2]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo")

# chain 연결 (LCEL)
chain = prompt | llm

# chain 호출
result = chain.invoke({"input": "인공지능 모델의 학습 원리"})
print(type(result))
print(result.content)

<class 'langchain_core.messages.ai.AIMessage'>
인공지능 모델의 학습 원리는 크게 두 가지 방법으로 나눌 수 있습니다. 첫 번째는 지도학습이며, 이는 레이블이 달린 데이터를 사용하여 모델을 학습시키는 방법입니다. 모델은 입력 데이터와 해당 입력 데이터에 대한 정확한 출력(레이블)을 학습하여 패턴을 식별하고 예측을 수행합니다. 

두 번째 방법은 비지도학습입니다. 이는 레이블이 달려있지 않은 데이터를 사용하여 모델을 학습시키는 방법입니다. 모델은 입력 데이터의 패턴을 식별하고 클러스터링, 차원 축소, 이상치 탐지 등의 작업을 수행합니다.

두 방법 모두 학습 과정은 데이터를 입력하고 모델이 예측한 출력 값과 실제 값 사이의 오차를 최소화하는 방향으로 가중치를 조정하면서 반복적으로 학습이 진행됩니다. 이를 통해 모델은 데이터의 패턴을 학습하고 새로운 입력에 대한 예측을 수행할 수 있습니다.


### LLM 체인
- ( Prompt + LLM 모델 + 출력파서) - LCEL

In [5]:
# 1. 프롬프트 생성
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

prompt = PromptTemplate.from_template(
    "You are an expert in AI Expert. Answer the question. <Question>: {input}에 대해 쉽게 설명해주세요."
)
prompt

PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='You are an expert in AI Expert. Answer the question. <Question>: {input}에 대해 쉽게 설명해주세요.')

In [6]:
llm = ChatOpenAI(model="gpt-3.5-turbo")

output_parser = StrOutputParser()

In [8]:
# 2. chain 생성 (LCEL)
chain = prompt | llm | output_parser

In [9]:
# 3. chain의 invoke 호출
result = chain.invoke({"input": "인공지능 모델의 학습 원리"})
print(type(result))
print(result)

<class 'str'>
<Answer>:
인공지능 모델의 학습원리는 데이터를 입력으로 받아 일련의 계산을 통해 예측을 수행하는 과정입니다. 모델은 입력 데이터와 정답 데이터 사이의 관계를 학습하여 새로운 데이터에 대해 예측을 할 수 있도록 학습됩니다. 학습 과정에서 모델은 오차를 최소화하기 위해 가중치를 조정하며, 이를 통해 데이터 간의 패턴과 관련성을 파악하게 됩니다. 이러한 학습과정을 통해 모델은 새로운 데이터에 대한 예측을 보다 정확하게 수행할 수 있게됩니다.


In [10]:
try:
    result = chain.invoke({"input": "LangChain의 Products(제품)는 어떤 것들이 있나요?"})
    print(type(result))
    print(result)
except Exception as e:
    print(f"오류 발생: {e}")

<class 'str'>
LangChain은 다양한 AI 기술과 서비스를 제공하는 기업입니다. LangChain의 제품에는 다국어 번역 및 통역 솔루션, 자연어 처리 기술을 활용한 챗봇 및 가상 비서 서비스, 언어 학습 및 교육 플랫폼 등이 있습니다. 또한 LangChain은 고객의 요구에 맞는 맞춤형 AI 솔루션을 개발하여 제공하고 있습니다.
